In [1]:
"""Tutorial: Overlap, Kinetic, and Dipole Integrals"""

__author__    = ["Justin M. Turney"]
__credit__    = ["Justin M. Turney"]

__copyright__ = "(c) 2014-2017, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2017-08-10"

# Overlap, Kinetic, and Dipole Integrals

In this tutorial we will compute the overlap and kinetic energy integrals encountered in Hartree-Fock. We will also compute the dipole integrals to obtain the molecular dipole moment. 

## Recurrence formula for one-electron integrals over Gaussian functions
Direct calculation of all integrals is not only complicated, but expensive. Here, we will implement an Obara-Saika recursion scheme for a simpler and more efficient implementation.

### Cartesian Gaussian functions
Denote the origin of a 3-dimensional cartesian gaussian function by the coordinates $\mathbf{R} = (R_x, R_y, R_z)$.
Let $\mathbf{r} = (x, y, z)$ be the coordinates of the electron, and $\alpha$ be the orbital exponent. We can now define an *unnormalized* cartesian gaussian function as:

\begin{equation}
\phi(\mathbf r; \alpha, \mathbf n, \mathbf R) = (x - R_x)^{n_x} (y - R_y)^{n_y} (z - R_z)^{n_z} \exp[-\alpha (\mathbf r - \mathbf R)^2]
\end{equation}

where $\alpha$ is the orbital exponent, and $\mathbf{n} = (n_x, n_y, n_z)$ is the angular momentum index vector. The sum $n_x + n_y + n_z = \lambda$ will hereafter be referred to as the **angular momentum**. We define a **shell** to be a set of functions (*components*) which share the same origin $\mathbf{R}$, angular momentum $\lambda$, and orbital exponent $\alpha$. 

The shells with $\lambda$  equal to $0, 1, 2,...,$ are referred to as the $s, p, d, ...$ shell. Each shell has $(\lambda + 1) (\lambda + 2)/2$ components. The $s$ shell, with angular momentum $\lambda = 0$ has one component usually designated as $s$. The $p$ shell ($\lambda = 1$) has three components, designated as $p_x, p_y, p_z$. The $d$ shell ($\lambda = 2$) has six components, designated as $d_{xx}, d_{yy}, d_{zz}, d_{xy}, d_{xz}, d_{yz}$.

Using our angular momentum index vector $\mathbf{n}$ we denote the single component of the $s$ shell to have angular momentum index $\mathbf{n} = \mathbf{0} = (0, 0, 0)$. Since the $p$ shell has three components, we may compactly express the angular momentum index vector as $\mathbf{1}_i$ where $i$ may be $x$, $y$, or $z$, and $\mathbf{1}_i = (\delta_{ix}, \delta_{iy}, \delta_{iz})$. For example, $p_x$ may be represented as $\mathbf{1}_x = (1, 0, 0)$. For the six components of the $d$ shell, we require a sum of two angular momentum index vectors $\mathbf{1}_i + \mathbf{1}_j$, where $(i,j = x,y,z)$. In this notation, the $d_{xy}$ component is $\mathbf{1}_x + \mathbf{1}_y = (1,0,0) + (0,1,0) = (1,1,0)$. To obtain higher order angular momentum components, we add the appropriate number of $\mathbf{1}_i$'s ($\mathbf{n}$'s)


In later discussions all the components  in a shell are assumed to be exhausted as the basic functions, and to be treated together in the computation of molecular integrals.


### Two-center overlap integrals
Two-center overlap integrals over unnormalized Cartesian Gaussian functions are of the form:
\begin{equation}
(\mathbf a|\mathbf b) = \int d\mathbf r\ \phi(\mathbf r; \alpha_a, \mathbf a, \mathbf A)\phi(\mathbf r; \alpha_b, \mathbf b, \mathbf B)
\end{equation}

given $(\mathbf 0_A | \mathbf 0_B)$, we can use the Obara-Saika recursion relation to obtain overlap integrals between all basis functions. The overlap over $s$ functions is given by:

\begin{equation}
(\mathbf 0_A | \mathbf 0_B) = \left(\frac{\pi}{\alpha}\right)^{3/2} \exp[-\xi(\mathbf A-\mathbf B)^2]
\end{equation}

where $\alpha = \alpha_a + \alpha_b$ and $\zeta = \frac{\alpha_a\alpha_b}{\alpha}$.
The recursion relations are given below. For a full derivation, see the appendix, or the original paper by Obara and Saika paper. To increment the left side angular momentum:

\begin{equation}
(\mathbf a+\mathbf 1_i|\mathbf b) = (\mathbf{P - A})(\mathbf a|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf a)(\mathbf a-\mathbf 1_i|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf b)(\mathbf a|\mathbf b-\mathbf 1_i)
\end{equation}
and similarily, to increment the right side:
\begin{equation}
(\mathbf a|\mathbf b+\mathbf 1_i) = (\mathbf{P - B})(\mathbf a|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf a)(\mathbf a-\mathbf 1_i|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf b)(\mathbf a|\mathbf b-\mathbf 1_i)
\end{equation}


where $\mathbf{P} = \frac{\alpha_a \mathbf{A} + \alpha_b \mathbf{B}} {\alpha}$, and ${N}_i$ is ???



### Write the recursion function
Write a function that performs the Obara-Saika recursion for a given set of 

The Obara-Saika recursion relationships depend on $\mathbf{P-A}$ and  $\mathbf{P-B}$, $\alpha$,  and two angular momenta (for $\mathbf a$ and $\mathbf b$) values which we will denote as `PA`, `PB`, `alpha`, `AMa`, and `AMb`. 
Let's write a function that takes these parameters and returns three matrices containing the x, y and z components of our unnormalized overlap integrals. These same components also can be used to construct our kinetic energy and dipole integrals later, as we will see.

For now, we will set $(\mathbf 0 | \mathbf 0)$ to $1.0$ and account for it later. 

In [2]:
import psi4
import numpy as np

def os_recursion(PA, PB, alpha, AMa, AMb):
    if len(PA) != 3 or len(PB) != 3:
        raise "PA and PB must be xyz coordinates."

    # Allocate space x, y, and z matrices
    # We add one because the equation for the kinetic energy
    # integrals require terms one beyond those in the overlap
    x = np.zeros((AMa + 1, AMb + 1))
    y = np.zeros((AMa + 1, AMb + 1))
    z = np.zeros((AMa + 1, AMb + 1))

    # Define 1/2alpha factor for convenience
    oo2a = 1.0 / (2.0 * alpha)

    # Set initial conditions (a|b) to 'start' for each component
    x[0, 0] = y[0, 0] = z[0, 0] = 1.0

    
    # BEGIN RECURSION
    # Upward recursion in b for a = 0
    # Fill in the [0,1] position with PB
    if AMb > 0:
        x[0, 1] = PB[0]
        y[0, 1] = PB[1]
        z[0, 1] = PB[2]

    # Fill in the rest of row zero
    for b in range(1, AMb):
        x[0, b + 1] = PB[0] * x[0, b] + b * oo2a * x[0, b - 1]
        y[0, b + 1] = PB[1] * y[0, b] + b * oo2a * y[0, b - 1]
        z[0, b + 1] = PB[2] * z[0, b] + b * oo2a * z[0, b - 1]

    # Upward recursion in a for all b's
    # Fill in the [1,0] position with PA
    if AMa > 0:
        x[1, 0] = PA[0]
        y[1, 0] = PA[1]
        z[1, 0] = PA[2]

        # Fill in the rest of row one
        for b in range(1, AMb + 1):
            x[1, b] = PA[0] * x[0, b] + b * oo2a * x[0, b - 1]
            y[1, b] = PA[1] * y[0, b] + b * oo2a * y[0, b - 1]
            z[1, b] = PA[2] * z[0, b] + b * oo2a * z[0, b - 1]

        # Fill in the rest of column 0
        for a in range(1, AMa):
            x[a + 1, 0] = PA[0] * x[a, 0] + a * oo2a * x[a - 1, 0]
            y[a + 1, 0] = PA[1] * y[a, 0] + a * oo2a * y[a - 1, 0]
            z[a + 1, 0] = PA[2] * z[a, 0] + a * oo2a * z[a - 1, 0]
    
        # Fill in the rest of the a'th row
            for b in range(1, AMb + 1):
                x[a + 1, b] = PA[0] * x[a, b] + a * oo2a * x[a - 1, b] + b * oo2a * x[a, b - 1]
                y[a + 1, b] = PA[1] * y[a, b] + a * oo2a * y[a - 1, b] + b * oo2a * y[a, b - 1]
                z[a + 1, b] = PA[2] * z[a, b] + a * oo2a * z[a - 1, b] + b * oo2a * z[a, b - 1]

    # Return the results
    return (x, y, z)


In [3]:
os_recursion([1.0, 2.0, 3.0], [3.0, 2.0, 1.0], 5.0, 3, 3)

(array([[  1.   ,   3.   ,   9.1  ,  27.9  ],
        [  1.   ,   3.1  ,   9.7  ,  30.63 ],
        [  1.1  ,   3.5  ,  11.23 ,  36.33 ],
        [  1.3  ,   4.23 ,  13.87 ,  45.825]]),
 array([[  1.   ,   2.   ,   4.1  ,   8.6  ],
        [  2.   ,   4.1  ,   8.6  ,  18.43 ],
        [  4.1  ,   8.6  ,  18.43 ,  40.3  ],
        [  8.6  ,  18.43 ,  40.3  ,  89.815]]),
 array([[  1.   ,   1.   ,   1.1  ,   1.3  ],
        [  3.   ,   3.1  ,   3.5  ,   4.23 ],
        [  9.1  ,   9.7  ,  11.23 ,  13.87 ],
        [ 27.9  ,  30.63 ,  36.33 ,  45.825]]))

# Overlap Integrals

Now that we have our recursion set up, we are ready to compute the integrals. First we need a molecule and a basis set. We compute a quick Hartree-Fock to obtain a wavefunction for easy access to the basis set, and we can compare Psi4's HF energy to our energy derived from our hand-coded integrals.

In [4]:
mol = psi4.geometry("""
                        O
                        H 1 1.1
                        H 1 1.1 2 104
                        symmetry c1
                        """)


psi4.set_options({'basis':        'sto-3g',
                  'scf_type':     'pk',
                  'mp2_type':     'conv',
                  'puream': 0,
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

scf_e, scf_wfn = psi4.energy('scf', return_wfn = True)

basis = scf_wfn.basisset()
mints = psi4.core.MintsHelper(basis)


The factored form of the two-center overlap integrals is
\begin{equation}
(\mathbf a | \mathbf b) = \exp[-\xi(\mathbf A-\mathbf B)^2]\ I_x(n_{ax},n_{bx},n_{cx})\ I_y(n_{ay},n_{by},n_{cy})\ I_z(n_{az},n_{bz},n_{cz})
\end{equation}

where I_x is the x component of the unnormalized overlap matrix and $\kappa_{ab}$ is  $\exp[-\xi (\mathbf A - \mathbf B)^2]$

To obtain the overlap matrix, we must loop over all shells, and for each shell, loop over the components (primitive gaussians). 

~~~python
# make space to store the overlap integral matrix
S = np.zeros((basis.nao(),basis.nao()))

# loop over the shells, basis.nshell is the number of shells
for i in range(basis.nshell()):
    for j in range(basis.nshell()):
        #basis.shell is a shell (1s, 2s, 2p, etc.)
        #for water, there are 5 shells: (H: 1s, H: 1s, O: 1s, 2s, 2p)
        ishell = basis.shell(i) 
        jshell = basis.shell(j)
        #each shell has some number of primitives which make up each component of a shell
        #sto-3g has 3 primitives for every component of every shell.
        nprimi = ishell.nprimitive 
        nprimj = jshell.nprimitive
       
~~~

at this point we access the orbital exponent $\alpha$ for each primitive basis set $\mathbf{a}$ and $\mathbf{b}$ in $(\mathbf{a}|\mathbf{b})$. The `basis.shell()` (which we have set to `ishell` and `jshell`) is a `psi4.core.GaussianShell` object, which contains all the information we need about the primitives within a shell, such as the exponent `(shell().exp())`, the coefficient `(shell().coef())`, the angular momentum `(shell().am)` and the atom number of a shell `(shell().ncenter)` which we can use to get the coordinates of the primitive center ($\mathbf{A}$ or $\mathbf{B}$) (whew!). We store all of these in our for-loops as variables for easy access:

~~~python
        #loop over the primitives within a shell
        for a in range(nprimi):  
            for b in range(nprimj):
                expa = ishell.exp(a) #exponents
                expb = jshell.exp(b)
                coefa = ishell.coef(a)  #coefficients
                coefb = jshell.coef(b)
                AMa = ishell.am  #angular momenta
                AMb = jshell.am
                #defining centers for each basis function 
                #mol.x() returns the x coordinate of the atom given by ishell.ncenter
                #we use this to define a coordinate vector for our centers
                A = [mol.x(ishell.ncenter), mol.y(ishell.ncenter), mol.z(ishell.ncenter)]
                B = [mol.x(jshell.ncenter), mol.y(jshell.ncenter), mol.z(jshell.ncenter)]
~~~

We are now ready to define all the parameters that go into our recursion function and define the intitial starting point `start` of our recursion which should be the $s$ orbital overlap:
\begin{equation} s = (\mathbf 0_A | \mathbf 0_B) = \left(\frac{\pi}{\alpha}\right)^{3/2} \exp[-\xi(\mathbf A-\mathbf B)^2] 
\end{equation}
                
                
~~~python
                alpha = expa + expb
                zeta = (expa * expb) / alpha
                P = (expa * A + expb * B) / alpha
                PA = P - A
                PB = P - B
                AB = A - B
                start = (np.pi / alpha)**(3 / 2) * np.exp(-zeta * (AB[0]**2 + AB[1]**2 + AB[2]**2))
                # call the recursion function
                x, y, z = os_recursion(PA, PB, alpha, AMa, AMb, start)
~~~


We are now ready to assign the elements of our overlap matrix. Currently, we are only looping over the shells, but not the individual components of each shell (e.g. we are not distinguishing between $p_x$, $p_y$, and $p_z$). We must keep track of our blah

~~~python
                counta = 0
                
                for ii in range(AMa + 1):
                    la = AMa - ii
                    for jj in range(ii + 1):
                        ma = ii - jj
                        na = jj
                        countb = 0
                        for aa in range(AMb + 1):
                            lb = AMb - aa
                            for bb in range(aa + 1):
                                mb = aa - bb
                                nb = bb
                                
                                S[ishell.function_index + counta, jshell.function_index + countb] += start
                                S[ishell.function_index + counta, jshell.function_index + countb] *= coefa
                                S[ishell.function_index + counta, jshell.function_index + countb] *= coefb
                                S[ishell.function_index + counta, jshell.function_index + countb] *= x[la,lb]
                                S[ishell.function_index + counta, jshell.function_index + countb] *= y[ma,mb]
                                S[ishell.function_index + counta, jshell.function_index + countb] *= z[na,nb]
~~~

Putting it all together we can form our full overlap matrix:

In [7]:
# make space to store the overlap integral matrix
S = np.zeros((basis.nao(),basis.nao()))

# loop over the shells, basis.nshell is the number of shells
for i in range(basis.nshell()):
    for j in range(basis.nshell()):
        #basis.shell is a shell (1s, 2s, 2p, etc.)
        #for water, there are 5 shells: (H: 1s, H: 1s, O: 1s, 2s, 2p)
        ishell = basis.shell(i) 
        jshell = basis.shell(j)
        #each shell has some number of primitives which make up each component of a shell
        #sto-3g has 3 primitives for every component of every shell.
        nprimi = ishell.nprimitive 
        nprimj = jshell.nprimitive
        #loop over the primitives within a shell
        for a in range(nprimi):  
            for b in range(nprimj):
                expa = ishell.exp(a) #exponents
                expb = jshell.exp(b)
                coefa = ishell.coef(a)  #coefficients
                coefb = jshell.coef(b)
                AMa = ishell.am  #angular momenta
                AMb = jshell.am
                #defining centers for each basis function 
                #mol.x() returns the x coordinate of the atom given by ishell.ncenter
                #we use this to define a coordinate vector for our centers
                A = np.array([mol.x(ishell.ncenter), mol.y(ishell.ncenter), mol.z(ishell.ncenter)])
                B = np.array([mol.x(jshell.ncenter), mol.y(jshell.ncenter), mol.z(jshell.ncenter)])

                alpha = expa + expb
                zeta = (expa * expb) / alpha
                P = (expa * A + expb * B) / alpha
                PA = P - A
                PB = P - B
                AB = A - B
                start = (np.pi / alpha)**(3 / 2) * np.exp(-zeta * (AB[0]**2 + AB[1]**2 + AB[2]**2))
               
                # call the recursion
                x, y, z = os_recursion(PA, PB, alpha, AMa+1, AMb+1)

                counta = 0
                for ii in range(AMa + 1):
                    la = AMa - ii
                    for jj in range(ii + 1):
                        ma = ii - jj
                        na = jj
                        countb = 0
                        for aa in range(AMb + 1):
                            lb = AMb - aa
                            for bb in range(aa + 1):
                                mb = aa - bb
                                nb = bb
                            

                                S[ishell.function_index + counta, jshell.function_index + countb] += start * coefa * coefb

                                S[ishell.function_index + counta, jshell.function_index + countb] *= x[la,lb]
                                S[ishell.function_index + counta, jshell.function_index + countb] *= y[ma,mb]
                                S[ishell.function_index + counta, jshell.function_index + countb] *= z[na,nb]
                                
                                #S[ishell.function_index + counta, jshell.function_index + countb] += start*coefa*coefb *x[la,lb]*y[ma,mb]*z[na,nb]
                                
                                #S[ishell.function_index + counta, jshell.function_index + countb] *= coefb*x[la,lb]*y[ma,mb]*z[na,nb]
                
                                countb += 1
                        counta += 1
                        

Check our overlap against Psi4:

In [8]:
Spsi4 = np.asarray(mints.ao_overlap())
print(S)
print(Spsi4)
np.allclose(S, Spsi4, 5)

[[ 1.          0.23670394  0.          0.          0.          0.03840559
   0.03840559]
 [ 0.23670394  1.          0.          0.          0.          0.38613879
   0.38613879]
 [ 0.          0.          0.26923584  0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.26923584  0.         -0.00190698
   0.14627216]
 [ 0.          0.          0.          0.          0.26923584  0.08872581
   0.08872581]
 [ 0.03840559  0.38613879  0.         -0.04054055  0.0562398   1.
   0.18175985]
 [ 0.03840559  0.38613879  0.          0.07636306  0.0562398   0.18175985
   1.        ]]
[[  1.00000000e+00   2.36703937e-01   0.00000000e+00   0.00000000e+00
   -1.61865880e-18   3.84055921e-02   3.84055921e-02]
 [  2.36703937e-01   1.00000000e+00   0.00000000e+00   0.00000000e+00
   -1.77144813e-17   3.86138791e-01   3.86138791e-01]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000

True

### Recurrence expressions for overlap integrals over $s$, $p$, and $d$  Cartesian Gaussian funcions
\begin{align}
(s|s) =& \left(\frac{\pi}{\alpha}\right)^{3/2} \exp\left\{-\xi(\mathbf A-\mathbf B)^2\right\} \\
(p_i|s) =& (P_i - A_i)(s|s) \\
(p_i|p_j) =& (P_j - B_j)(p_i|s) + \frac{\delta_{ij}}{2\alpha}(s|s) \\
(d_{ij}|s) =& (P_j-A_j)(p_i|s) + \frac{\delta_{ij}}{2\alpha}(s|s) \\
(d_{ij}|p_k) =& (P_k-B_k)(d_{ij}|s) + \frac{\delta_{ik}}{2\alpha}(p_j|s) + \frac{\delta_{jk}}{2\alpha}(p_i|s) \\
(d_{ij}|d_{kl}) =& (P_l-B_l)(d_{ij}|p_k) + \frac{\delta_{il}}{2\alpha}(p_j|p_k) + \frac{\delta_{jl}}{2\alpha}(p_i|p_k) + \frac{\delta_{kl}}{2\alpha}(d_{ij}|s) \\
\\
& (i,j,k,l = x,y,z)
\end{align}

### Cartesian function ordering that Psi4 uses
In writing your recursion, you will need to be aware of how Psi4 orders Cartesian functions with higher angular momentum:
\begin{align}
\mathbf s &= s \\
\mathbf p &= p_x, p_y, p_z \\
\mathbf d &= d_{x^2}, d_{xy}, d_{xz}, d_{y^2}, d_{yz}, d_{z^2} \\
\mathbf f &= f_{x^3}, f_{x^2y}, f_{x^2z}, f_{xy^2}, f_{xyz}, f_{xz^2}, f_{y^3}, f_{y^2z}, f_{yz^2}, f_{z^3}
\end{align}



## One electron integrals
Now it is time to compute the following one-electron integrals:
- overlap
- kinetic
- dipole moment

Due to the added complexity of the nuclear attraction integrals and the two-electron repulsion integrals, we will not compute them in this tutorial.

### Kinetic energy integrals 
Before proceeding to an analysis of the kinetic energy integral, it will prove convenient to establish a short hard notation for integrals related to the overlap integral:
\begin{equation}
(0|0) = \int G(\alpha_1, \mathbf A, l_1, m_1, n_1) G(\alpha_2, \mathbf B, l_2, m_2, n_2)\ d\tau
\end{equation}
The symbol $<+\mathrm{n}|0>_{\mathrm{x}}$ will denote an integral of the form given by the above equation, except that the quantum number $l_1$ has been incremented by n. Similar notation applies to the $m_1$ and $n_1$ quantum numbers with the use of subscripts x and y.  We will also use the symbols $<0|+\mathrm{n}>_{\mathrm{x}}$, where we have incremented the quantum number $l_2$ by n. You should see the pattern.

The kinetic energy integral is defined as
\begin{equation}
T_{ij} = -\frac{1}{2} \int \phi_i(1) \nabla_1^2 \phi_j(1)\ d\tau_1
\end{equation}
and as
\begin{equation}
KE = - \frac{1}{2}  \int G(\alpha_1, \mathbf A, l_1, m_1, n_1) \left(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}\right) G(\alpha_2, \mathbf B, l_2, m_2, n_2)\ d\tau
\end{equation}

Evaluate the kinetic energy integrals for $I_x$, $I_y$, and $I_z$ and code them up to obtain $T_{ij}$. The equations that you obtain for these integrals are "unsymmetric" because the quantum numbers of the Gaussian-type function centered on $\mathbf B$ are altered while those of the Gaussian-type function on $\mathbf A$ are not.

Here is the symmetric form of the kinetic energy integral $I_x$ component.
\begin{equation}
I_x = \frac{1}{2}\Big\{l_1 l_2 (-1|-1)_{\mathrm{x}} + 4\alpha_1 \alpha_2 (+1|+1)_{\mathrm{x}} - 2\alpha_1 l_2 (+1|-1)_{\mathrm{x}} - 2\alpha_2 l_1 (-1|+1)_{\mathrm{x}}\Big\}
\end{equation}

### Dipole moment integrals
We discuss these non-energy integrals here because they are frequently used and closely connected with the overlap and kinetic energy integrals. The dipole moment is defined with respect to a point in space $\mathbf C$, which is almost always taken to be the center of mass. Fortunately for our purposes, Psi4 automatically moves the molecule to be centered on the center of mass and thus $\mathbf C$ is $\mathbf 0$. The dipole moment integral is written for the $x$ direction as
\begin{equation}
d_{\mathrm{x}} =  \int G(\alpha_1, \mathbf A, l_1, m_1, n_1) G(\alpha_2, \mathbf B, l_2, m_2, n_2) \mathrm{x_c}\ d\tau
\end{equation}
and similarly for the operators $\mathrm{y_c}$ and $\mathrm{z_c}$. A convenient procedure is to redefine $\mathrm{x_c}$ in terms of  $\mathrm{x_A}$ or $\mathrm{x_B}$, and we will use $\mathrm{x_A}$. Thus
\begin{equation}
\mathrm{x_c} = \mathrm{x} - \mathrm{C_x} = (\mathrm{x - A_x}) + (\mathrm{A_x - C_x}) = \mathrm{x_A + \mathbf {AC}_x}
\end{equation}
and we find
\begin{equation}
d_\mathrm{x} = (+1|0)_{\mathrm{x}} + \mathbf{AC}_{\mathrm{x}} (0|0)
\end{equation}
Similarily for $d_{\mathrm{y}}$ and $d_{\mathrm{z}}$.

#### The x component
\begin{align}
\mu_x =& - \mu_{\mathrm{elec}} + \mu_{\mathrm{nuclear}}\\
\mu_x =& - \sum_{\mu \nu}^{\mathrm{AO}} D_{\mu \nu} d_{\mu \nu}^{x} + \sum_A^N Z_A X_A
\end{align}

You can obtain the nuclear contribution to the dipole $\mu_{\mathrm{nuclear}}$ from Psi4 using

    psi4.core.nuclear_dipole(molecule).to_array()

#### The total dipole moment
\begin{equation}
\mu = \sqrt{\mu_x^2 + \mu_y^2 + \mu_z^2}
\end{equation}

### Kinetic energy integrals 
Before proceeding to an analysis of the kinetic energy integral, it will prove convenient to establish a short hard notation for integrals related to the overlap integral:
\begin{equation}
(0|0) = \int G(\alpha_1, \mathbf A, l_1, m_1, n_1) G(\alpha_2, \mathbf B, l_2, m_2, n_2)\ d\tau
\end{equation}
The symbol $<+\mathrm{n}|0>_{\mathrm{x}}$ will denote an integral of the form given by the above equation, except that the quantum number $l_1$ has been incremented by n. Similar notation applies to the $m_1$ and $n_1$ quantum numbers with the use of subscripts x and y.  We will also use the symbols $<0|+\mathrm{n}>_{\mathrm{x}}$, where we have incremented the quantum number $l_2$ by n. You should see the pattern.

The kinetic energy integral is defined as
\begin{equation}
T_{ij} = -\frac{1}{2} \int \phi_i(1) \nabla_1^2 \phi_j(1)\ d\tau_1
\end{equation}
and as
\begin{equation}
KE = - \frac{1}{2}  \int G(\alpha_1, \mathbf A, l_1, m_1, n_1) \left(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}\right) G(\alpha_2, \mathbf B, l_2, m_2, n_2)\ d\tau
\end{equation}

Evaluate the kinetic energy integrals for $I_x$, $I_y$, and $I_z$ and code them up to obtain $T_{ij}$. The equations that you obtain for these integrals are "unsymmetric" because the quantum numbers of the Gaussian-type function centered on $\mathbf B$ are altered while those of the Gaussian-type function on $\mathbf A$ are not.

Here is the symmetric form of the kinetic energy integral $I_x$ component.
\begin{equation}
I_x = \frac{1}{2}\Big\{l_1 l_2 (-1|-1)_{\mathrm{x}} + 4\alpha_1 \alpha_2 (+1|+1)_{\mathrm{x}} - 2\alpha_1 l_2 (+1|-1)_{\mathrm{x}} - 2\alpha_2 l_1 (-1|+1)_{\mathrm{x}}\Big\}
\end{equation}


### Dipole moment integrals
We discuss these non-energy integrals here because they are frequently used and closely connected with the overlap and kinetic energy integrals. The dipole moment is defined with respect to a point in space $\mathbf C$, which is almost always taken to be the center of mass. Fortunately for our purposes, Psi4 automatically moves the molecule to be centered on the center of mass and thus $\mathbf C$ is $\mathbf 0$. The dipole moment integral is written for the $x$ direction as
\begin{equation}
d_{\mathrm{x}} =  \int G(\alpha_1, \mathbf A, l_1, m_1, n_1) G(\alpha_2, \mathbf B, l_2, m_2, n_2) \mathrm{x_c}\ d\tau
\end{equation}
and similarly for the operators $\mathrm{y_c}$ and $\mathrm{z_c}$. A convenient procedure is to redefine $\mathrm{x_c}$ in terms of  $\mathrm{x_A}$ or $\mathrm{x_B}$, and we will use $\mathrm{x_A}$. Thus
\begin{equation}
\mathrm{x_c} = \mathrm{x} - \mathrm{C_x} = (\mathrm{x - A_x}) + (\mathrm{A_x - C_x}) = \mathrm{x_A + \mathbf {AC}_x}
\end{equation}
and we find
\begin{equation}
d_\mathrm{x} = (+1|0)_{\mathrm{x}} + \mathbf{AC}_{\mathrm{x}} (0|0)
\end{equation}
Similarily for $d_{\mathrm{y}}$ and $d_{\mathrm{z}}$.

#### The x component
\begin{align}
\mu_x =& - \mu_{\mathrm{elec}} + \mu_{\mathrm{nuclear}}\\
\mu_x =& - \sum_{\mu \nu}^{\mathrm{AO}} D_{\mu \nu} d_{\mu \nu}^{x} + \sum_A^N Z_A X_A
\end{align}

You can obtain the nuclear contribution to the dipole $\mu_{\mathrm{nuclear}}$ from Psi4 using

    psi4.core.nuclear_dipole(molecule).to_array()

#### The total dipole moment
\begin{equation}
\mu = \sqrt{\mu_x^2 + \mu_y^2 + \mu_z^2}
\end{equation}

## Load Psi4 and NumPy
As with the Hartree-Fock tutorial, we'll start by importing some useful modules and setting basic quantities for use in Psi4.

In [78]:
import psi4
import numpy as np

# I'll show some plots
import matplotlib.pyplot as plt

# Load in the required options
import configparser
config = configparser.ConfigParser()
config.read('Options.ini')

molecule = psi4.geometry(config['DEFAULT']['molecule'])

# For your integrals code to match Psi4's ordering you must include "puream=0"
basis = psi4.core.BasisSet.build(molecule, 'BASIS', 'STO-3G', puream=0)
mints = psi4.core.MintsHelper(basis)

KeyError: 'molecule'

## Determine some information about the basis set

In [ ]:
# Look at the help documentation for the basis set
help(basis)
help(psi4.core.GaussianShell)

In [ ]:
print('The number of shells: %d' % (basis.nshell()))
print('The number of basis functions: %d' % (basis.nao()))

In [ ]:
# Print out some information about the shells in the basis set
for ishell in range(basis.nshell()):
    shell = basis.shell(ishell)
    print('Shell %d:' % (ishell))
    print('  Atom: %d' % (shell.ncenter))
    print('  AM: %d' % (shell.am))
    print('  # Cartesian functions: %d' % (shell.ncartesian))
    print('  # of primitive Gaussians: %d ' % (shell.nprimitive))
    print('  function_index: %d' % (shell.function_index))
    print('  center: %f, %f, %f' % (molecule.x(shell.ncenter), molecule.y(shell.ncenter), molecule.z(shell.ncenter)))

### Let's plot some of the functions to see what they look like

In [ ]:
# This is to show you have to access individual shells and access their coefficients and exponents.
shell0 = basis.shell(0)

# generate a list of points to evaluate the basis function (R=0 to 10 in 100 points)
R = np.linspace(0, 2, 100, endpoint=True)

# Plot each primitive
for idx in range(shell0.nprimitive):
    v = shell0.coef(idx) * np.exp(-shell0.exp(idx) * R**2)
    plt.plot(R, v)
plt.show()

### Sample x, y, and z matrices

In [ ]:
import OS

In [ ]:
(x, y, z) = OS.os_recursion([1.0, 2.0, 3.0], [3.0, 2.0, 1.0], 1.0, 3, 3)
print('x=\n', x)
print('y=\n', y)
print('z=\n', z)

In [ ]:
dipole = [x.to_array() for x in mints.ao_dipole()]
print(dipole)
psi4.core.nuclear_dipole(molecule).to_array()

### Compute one-electron integrals
Now you should be ready to compute some integrals!

In [ ]:
import psi4
import numpy as np
from OS import os_recursion
# linalg package for matrix square root function:
from scipy import linalg as la
from collections import namedtuple
RecursionResults = namedtuple('RecursionResults', ['x', 'y', 'z'])

import configparser
config = configparser.ConfigParser()
config.read('Options.ini')

molecule = psi4.geometry(config['DEFAULT']['molecule'])
molecule.update_geometry()
basis = psi4.core.BasisSet.build(molecule, "BASIS", config['DEFAULT']['basis'], puream=0)
#This line is just for the two-electron integrals:
mints = psi4.core.MintsHelper(basis)

###################################
# Integral stuff
###################################
#Total number of AOs for molecule
naos = basis.nao()
#Matrix to store overlap integrals
S = np.zeros((naos, naos))
#Matrix to store kinetic energy integrals
K = np.zeros((naos, naos))
#Matrices to store dipole integrals
dipolex = np.zeros((naos, naos))
dipoley = np.zeros((naos, naos))
dipolez = np.zeros((naos, naos))

#Point with respect to which we
#define dipole moment
C = (0., 0., 0.)

for i in range(basis.nshell()): 
    for j in range(basis.nshell()):
        ishell = basis.shell(i)
        jshell = basis.shell(j)
        for p in range(ishell.nprimitive):
            for q in range(jshell.nprimitive):
                #p and q label primitives
                # l + m + n = angmom of shell
                AMa = ishell.am 
                AMb = jshell.am
        κab=ξ=        #
                Ax = molecule.x(ishell.ncenter)
                Ay = molecule.y(ishell.ncenter)
                Az = molecule.z(ishell.ncenter)
                Bx = molecule.x(jshell.ncenter)
                By = molecule.y(jshell.ncenter)
                Bz = molecule.z(jshell.ncenter)
                #alphas are exponents
                alphaa = ishell.exp(p)
                alphab = jshell.exp(q)
                #
                coefa = ishell.coef(p)
                coefb = jshell.coef(q)
                #
                alpha = alphaa + alphab
                Px = (alphaa*Ax + alphab*Bx)/(alpha)
                Py = (alphaa*Ay + alphab*By)/(alpha)
                Pz = (alphaa*Az + alphab*Bz)/(alpha)
                #
                PAx = Px - Ax
                PAy = Py - Ay
                PAz = Pz - Az
                # 
                PBx = Px - Bx
                PBy = Py - By
                PBz = Pz - Bz
                # 
                PA = (PAx, PAy, PAz)
                PB = (PBx, PBy, PBz)
                # 
                zeta = (alphaa*alphab)/(alpha)
                #
                kappa = (np.pi/alpha)**(3/2)*np.exp(-zeta*(Ax-Bx)**2)*np.exp(-zeta*(Ay-By)**2)*np.exp(-zeta*(Az-Bz)**2)
                #
                matx, maty, matz = os_recursion(PA, PB, alpha, AMa, AMb)
                #
                # Counter!!!
                counta = 0

                # l on atom one
                for k in range(AMa+1):
                    lo = AMa-k
                    # m on atom one
                    for l in range(AMa+1-lo):
                        mo = AMa - lo - l
                        # n on atom one
                        no = AMa - lo - mo
                        #
                        countb = 0
                        # l on atom two
                        for m in range(AMb+1):
                            lt = AMb-m
                            # m on atom two
                            for n in range(AMb+1-lt):
                                mt = AMb - lt - n
                                # n on atom two
                                nt = AMb - lt - mt
                                #print('AMb', lt, mt, nt)
                                # Set the matrix elements
                                S[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*maty[mo,mt]*matz[no,nt]
                                #
                                # Kinetic energy integral stuff
                                Tx = (1/2)*(lo*lt*matx[lo-1,lt-1] + 4*alphaa*alphab*matx[lo+1,lt+1] - 2*alphaa*lt*matx[lo+1,lt-1] - 2*alphab*lo*matx[lo-1,lt+1])
                                Ty = (1/2)*(mo*mt*maty[mo-1,mt-1] + 4*alphaa*alphab*maty[mo+1,mt+1] - 2*alphaa*mt*maty[mo+1,mt-1] - 2*alphab*mo*maty[mo-1,mt+1])
                                Tz = (1/2)*(no*nt*matz[no-1,nt-1] + 4*alphaa*alphab*matz[no+1,nt+1] - 2*alphaa*nt*matz[no+1,nt-1] - 2*alphab*no*matz[no-1,nt+1])
                                #
                                K[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*Tx*maty[mo,mt]*matz[no,nt] 
                                K[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*Ty*matz[no,nt] 
                                K[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*maty[mo,mt]*Tz
                                # 
                                dipolex[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*(matx[lo+1,lt] + (Ax)*matx[lo,lt])*maty[mo,mt]*matz[no,nt]
                                dipoley[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*(maty[mo+1,mt] + (Ay)*maty[mo,mt])*matz[no,nt]
                                dipolez[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*maty[mo,mt]*(matz[no+1,nt] + (Az)*matz[no,nt])
                                #
                                countb = countb + 1
                        #
                        counta = counta + 1


#### Compare to Psi4
Check your energy and dipole moment against the energy and dipole moment Psi4 outputs: